# Classic models - Explorando una BD MySQL con Python y Pandas
---

### Importando librerías necesarías

In [ ]:
### !pip install mysql.connector  <-- NO se recomienda esta forma!

# Usar mejor esta manera!
#     Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install mysql.connector

In [ ]:
import pandas as pd
import mysql.connector

### Estableciendo la conexión a la BD en MySQL Server

In [ ]:
# acceso a datos de user y pass de la BD
user_name = 'root'
user_pwd = 'admin'

In [ ]:
# conectando a la BD ClassicModels   se puede descargar de: http://mysql.tutorial.org
myconnection = mysql.connector.connect( host="localhost",
                                      user = user_name,
                                      passwd = user_pwd,
                                      db= 'classicmodels',
                                      port = 3306)
myconnection

### Listando las tablas de la BD

In [ ]:
lista_tablas = pd.read_sql_query("SHOW TABLES FROM  classicmodels", myconnection)
lista_tablas

### mostrando el esquema de cada tabla

In [ ]:
nombre_tablas = lista_tablas['Tables_in_classicmodels']

for quetabla in nombre_tablas:
    despliegue = pd.read_sql_query('DESCRIBE {}' . format(quetabla), myconnection)
    print("**** Tabla: " + quetabla)
    print(despliegue,'\n')


### Generando archivos CSV de salida por cada tabla de la BD

In [ ]:
#Cambiar localizacion de archivo antes de usar
path ="C:/Users/oskr/Desktop/PythonCSV/"
for quetabla in nombre_tablas:
    dataset = pd.read_sql_query('SELECT * FROM {}' . format(quetabla), myconnection)
    print("**** Grabando Tabla: " + quetabla)
    dataset.to_csv( path + quetabla + ".csv")
    print("   ....")


### Cargando los Datasets CSV como DataFrames

In [ ]:
customers = pd.read_csv(path + "customers.csv")
employees = pd.read_csv(path + "employees.csv")
offices = pd.read_csv(path + "offices.csv")
orders = pd.read_csv(path + "orders.csv")
orderdetails = pd.read_csv(path + "orderdetails.csv")
products = pd.read_csv(path + "products.csv")
payments = pd.read_csv(path + "payments.csv")

# se omite df de productlines y saldos

customers.head()


## Explorando los datos con SQL y con DF de Pandas


In [ ]:
customers.tail(5)

### SELECT * FROM <<tabla>>

In [ ]:
querySQL = "SELECT * FROM orders"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
salidaDF = orders
salidaDF


### SELECT * FROM ____ LIMIT 0,10

In [ ]:
querySQL = "SELECT * FROM customers LIMIT 0,10"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
salidaDF = customers[0:10]
salidaDF



---
### SELECT campo1, campo2 FROM ___

In [ ]:
querySQL = "SELECT customerNumber,customerName,city,country FROM customers "

salidaDF = customers[["customerNumber","customerName","city","country"]]
salidaDF



### SELECT __ FROM __ WHERE campo ISNULL

In [ ]:
querySQL = "SELECT * FROM orders WHERE comments IS NOT NULL "


#salidaDF = orders[orders["comments"].isnull()]
#salidaDF = orders[orders["comments"].notnull()]
salidaDF = orders[orders["comments"].isna()]
salidaDF



### SELECT DISTINCT campo FROM tabla

In [ ]:
# version SQL
querySQL = "SELECT DISTINCT productline FROM products"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
# version Pandas
salidaDF = products["productLine"].drop_duplicates()
salidaDF


## Operaciones condicionales y LIKE
###  SELECT ___ FROM ___ WHERE ___ BETWWEEN ___

In [ ]:
# version SQL
querySQL = "SELECT * FROM ORDERS WHERE orderDate BETWEEN '2003-01-01' AND '2003-01-10'"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
#version Pandas
orders.head()
salidaDF = orders[  (orders["orderDate"]>='2003-01-01') & (orders["orderDate"]<='2003-01-10')   ]
salidaDF




### SELECT __ FROM ___ WHERE campo LIKE 'expresion'

In [ ]:
querySQL = "SELECT * FROM customers WHERE customerName LIKE 'ba%'"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
#version Pandas
customers.head()
salidaDF = customers[   (customers["customerName"].str.startswith('Ba') )   ]   ##  cuidado es caseSensitive
salidaDF



In [ ]:
querySQL = "SELECT * FROM customers WHERE customerName LIKE '%Gift%'"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
#version Pandas
customers.head()
salidaDF = customers[(customers["customerName"].str.contains('Gift') ) ]   ##  cuidado es caseSensitive
salidaDF


### Ordenamiento de resultados

In [ ]:
# version SQL
querySQL = "SELECT * FROM products ORDER BY productLine, quantityInStock DESC"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
#version Pandas
products.head()
salidaDF = products.sort_values(["productLine","quantityInStock"],ascending=False )
salidaDF

###  Agrupamientos  GROUP BY
--- 

In [ ]:
#version Pandas
products.head()
salidaDF = products.groupby("productLine").size()   ### **** No se usa el .count()  ****
salidaDF


### JOINs

In [ ]:
# Hacer el INNER JOIN entre clientes y respectivos pedidos


#con Pandas
salidaDF = pd.merge(customers,orders,on="customerNumber")
salidaDF

In [ ]:
#  registros de clientesypedidos con fechaEnvio NULA
salidaDF =  salidaDF[ salidaDF["shippedDate"].isnull() ]
salidaDF




In [ ]:
# myconnection.close()


## Queries adicionales
---

- **Q1. Listado de datos de clientes y cantidad de pedidos.**
```   
     Columnas:  [CustomerNumber, CustomerName, City,Country, CantPedidos]
```
    
- **Q2. Listado de productos ordenados por categoria y cantidad de piezas pedidas.**
```
     Columnas:  [ProductCode, ProductName, ProductLine, sumaQuantityOrdered]
```
    
- **Q3. Promedio de PreciodeVenta y Cantidad de Piezas pedidas de cada uno de los productos.**
```    
     Columnas:  [ProductCode, ProductName, avgPrice, avgQuantityOrdered] 
```
    
- **Q4. Listado de datos de clientes con categorización de acuerdo a su CreditLimit.**
```   
     Categoría: "Platinum"(>=50000), "Gold"(>=25000<50000), "Silver"(>0<25000) y "Laton"(=0)
     Columnas:  [CustomerNumber, CustomerName, City, Country, CreditLimit, Categoría] 
```
        

# Queries con SQL

In [ ]:
querySQL = "SELECT TipoCliente, customerNumber, customerName, city, country, creditLimit FROM clientesa GROUP BY TipoCliente ORDER BY 1 ASC;"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
querySQL = "select p.productCode, p.productName, p.productLine, o.quantityOrdered from products as p left join orderdetails as o on o.productCode = p.productCode order by p.productLine, o.quantityOrdered ASC;"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
querySQL = "select p.productCode, p.productName, avg(o.priceEach) as AvgPrecio, avg(o.quantityOrdered) as AvgCantidad from products as p left join orderdetails as o on o.productCode = p.productCode group by p.productName;"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

In [ ]:
querySQL = "SELECT TipoCliente, customerNumber, customerName, city, country, creditLimit FROM clientesa GROUP BY TipoCliente ORDER BY 1 ASC;"
salidaSQL = pd.read_sql_query(querySQL, myconnection)
salidaSQL

# Queries con Pandas

In [ ]:
# code of query 1
dataframe_join = pd.merge(customers,orders,on="customerNumber")
# salidaDF = dataframe_join.groupby("customerNumber").size()
salidaDF = dataframe_join.groupby( ["customerNumber", "customerName", "city", "country"] ).size()
salidaDF

In [ ]:
query = orders[["customerNumber", "orderNumber"]
               ].groupby("customerNumber").count()
query = query.rename(columns={"orderNumber": "CanPedidos"})
salidaDF = pd.merge(customers, query, on="customerNumber")

salidaDF[["customerNumber", "customerName", "city", "country", "CanPedidos"]]


In [ ]:
prod = products[["productName", "productCode", "productLine"]]
ordet = orderdetails[["productCode", "quantityOrdered"]
                     ].groupby("productCode").sum("quantityOrdered")

salidaDF = pd.merge(prod, ordet, on="productCode")
salidaDF = salidaDF.sort_values(by=["productLine", "quantityOrdered"])

salidaDF


In [ ]:
def get_categoria(x):
    if x >=50000:
        return "Platinum"
    elif x >=25000 and x <50000:
        return "Gold"
    elif x >0 and x <25000:
        return "Silver"
    elif x == 0:
        return "Laton"
    
query4 = customers
query4["categoria"] = customers["creditLimit"].apply(lambda x: get_categoria)
query4

In [ ]:
prod = products[["productCode", "productName"]]
order = orderdetails[["productCode", "quantityOrdered", "priceEach"]]
total = order.assign(PromedioPVenta=order.quantityOrdered * order.priceEach)

avg = total[["productCode", "PromedioPVenta",
             "quantityOrdered"]].groupby("productCode").mean()
avg = avg.rename(columns={"quantityOrdered": "PromedioCPedida"})

salidaDF = pd.merge(prod, avg, on="productCode")
salidaDF


In [ ]:

def category(row):
    if int(row["creditLimit"]) >= 50000:
        result = "Platinum"
    elif int(row["creditLimit"]) >= 25000 < 50000:
        result = "Gold"
    elif int(row["creditLimit"]) > 0 < 25000:
        result = "Silver"
    else:
        result = "Laton"
    return result


customers["categoria"] = customers.apply(category, axis=1)
salidaDF = customers[["customerNumber", "customerName",
                      "city", "country", "creditLimit", "categoria"]]
salidaDF


### Ex 1

In [ ]:
pedido = 10395

query = pd.merge(products,orderdetails, on="productCode")

query = query[ query["orderNumber"]==pedido]

ordnumb = query[["orderNumber","productLine"]]

cantidad = query.groupby("productLine").count()["productName"]

query = query.assign(monto=query.priceEach*query.quantityOrdered).groupby("productLine").sum()[["quantityOrdered","monto"]]

basicinfo = pd.merge(query,cantidad,on="productLine")

result = pd.merge(basicinfo,ordnumb,on="productLine").drop_duplicates()

result


In [ ]:
customerss = pd.merge(customers,orders, on = "customerNumber")

customerss = customerss.rename(columns={"salesRepEmployeeNumber":"employeeNumber"})

employeess = pd.merge(customerss,employees, on = "employeeNumber")

resultt = pd.merge(offices,employeess,on="officeCode")


numPedidos = resultt.groupby("officeCode").count()["country_x"]
numCA = employeess[employeess["officeCode"]==employeess["officeCode"]].drop_duplicates(["customerNumber"]).groupby("officeCode").count()["shippedDate"]
resultt = resultt[["officeCode","city_x"]].drop_duplicates()

query2 = pd.merge(numPedidos,numCA, on="officeCode")
query2 = query2.rename(columns={"country_x":"total","shippedDate":"numAt"})

query2 = pd.merge(query2,resultt, on ="officeCode")
query2.sort_values(by=["total"],ascending=False)